In [1]:
# ==========================================================
# 📊 Graficador de Planos 3D: Intersección de tres planos 📐
# ==========================================================
# Autor: Martin Verastegui
# Fecha: 2024-12-01
# GitHub: https://github.com/goldhood
# Descripción:
# Este script permite ingresar tres ecuaciones de planos en 3D y visualizar sus
# intersecciones en un gráfico tridimensional. Se pueden observar los planos, sus
# puntos de intersección y las líneas donde se cruzan. Ideal para entender cómo
# interactúan los planos en el espacio.
# ==========================================================

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sympy import symbols, Eq, solve, linsolve
import ipywidgets as widgets
from IPython.display import display, clear_output

# Crear widgets para ingresar los coeficientes de las ecuaciones de manera más clara
# Ecuación 1: a1 * x + b1 * y + c1 * z = d1
equation1_box = widgets.HBox([
    widgets.Label(value="a₁*x + b₁*y + c₁*z = d₁:", layout=widgets.Layout(width='200px')),
    widgets.FloatText(value=1, description='a₁', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=2, description='b₁', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=3, description='c₁', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=10, description='d₁', layout=widgets.Layout(width='150px', height='40px'))
])

# Ecuación 2: a2 * x + b2 * y + c2 * z = d2
equation2_box = widgets.HBox([
    widgets.Label(value="a₂*x + b₂*y + c₂*z = d₂:", layout=widgets.Layout(width='200px')),
    widgets.FloatText(value=2, description='a₂', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=-1, description='b₂', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=1, description='c₂', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=5, description='d₂', layout=widgets.Layout(width='150px', height='40px'))
])

# Ecuación 3: a3 * x + b3 * y + c3 * z = d3
equation3_box = widgets.HBox([
    widgets.Label(value="a₃*x + b₃*y + c₃*z = d₃:", layout=widgets.Layout(width='200px')),
    widgets.FloatText(value=1, description='a₃', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=-1, description='b₃', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=-1, description='c₃', layout=widgets.Layout(width='150px', height='40px')),
    widgets.FloatText(value=-3, description='d₃', layout=widgets.Layout(width='150px', height='40px'))
])

# Botón para graficar
plot_button = widgets.Button(description="Graficar", button_style='primary', layout=widgets.Layout(width='200px', height='50px'))
rotation_slider = widgets.IntSlider(value=45, min=0, max=360, step=1, description='Rotación:', layout=widgets.Layout(width='400px'))
output = widgets.Output()

# Función para graficar los planos
def plot_planes(b=None):
    with output:
        clear_output(wait=True)

        # Obtener los valores ingresados por el usuario
        a1, b1, c1, d1 = [widget.value for widget in equation1_box.children[1:]]
        a2, b2, c2, d2 = [widget.value for widget in equation2_box.children[1:]]
        a3, b3, c3, d3 = [widget.value for widget in equation3_box.children[1:]]

        # Crear las mallas para graficar los planos
        x = np.linspace(-10, 10, 100)
        y = np.linspace(-10, 10, 100)
        X, Y = np.meshgrid(x, y)

        # Definir las funciones Z de cada plano
        Z1 = (d1 - a1 * X - b1 * Y) / c1
        Z2 = (d2 - a2 * X - b2 * Y) / c2
        Z3 = (d3 - a3 * X - b3 * Y) / c3

        # Crear la figura en 3D
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Graficar cada plano
        ax.plot_surface(X, Y, Z1, alpha=0.5, rstride=100, cstride=100, color='blue', label='Plano 1')
        ax.plot_surface(X, Y, Z2, alpha=0.5, rstride=100, cstride=100, color='red', label='Plano 2')
        ax.plot_surface(X, Y, Z3, alpha=0.5, rstride=100, cstride=100, color='green', label='Plano 3')

        # Etiquetas de los ejes
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title('Intersección de tres planos en el espacio')

        # Configurar la rotación del gráfico
        ax.view_init(elev=30, azim=rotation_slider.value)

        # Resolver las ecuaciones para encontrar las intersecciones entre los planos
        x_sym, y_sym, z_sym = symbols('x y z')
        eq1 = Eq(a1 * x_sym + b1 * y_sym + c1 * z_sym, d1)
        eq2 = Eq(a2 * x_sym + b2 * y_sym + c2 * z_sym, d2)
        eq3 = Eq(a3 * x_sym + b3 * y_sym + c3 * z_sym, d3)

        # Encontrar las intersecciones entre los pares de planos
        intersection_12 = linsolve([eq1, eq2], (x_sym, y_sym, z_sym))
        intersection_13 = linsolve([eq1, eq3], (x_sym, y_sym, z_sym))
        intersection_23 = linsolve([eq2, eq3], (x_sym, y_sym, z_sym))

        # Función para trazar la línea de intersección si existe
        def plot_intersection_line(solution, label, color):
            if solution:
                sol = list(solution)[0]
                if len(sol.free_symbols) > 0:
                    t = np.linspace(-10, 10, 100)
                    x_line = [float(sol[0].subs({list(sol.free_symbols)[0]: t_i})) for t_i in t]
                    y_line = [float(sol[1].subs({list(sol.free_symbols)[0]: t_i})) for t_i in t]
                    z_line = [float(sol[2].subs({list(sol.free_symbols)[0]: t_i})) for t_i in t]
                    ax.plot(x_line, y_line, z_line, color=color, label=label, linewidth=2)

        # Graficar las líneas de intersección
        plot_intersection_line(intersection_12, 'Intersección Plano 1 y 2', 'purple')
        plot_intersection_line(intersection_13, 'Intersección Plano 1 y 3', 'orange')
        plot_intersection_line(intersection_23, 'Intersección Plano 2 y 3', 'brown')

        # Resolver las ecuaciones para encontrar la intersección de los tres planos
        solution = linsolve([eq1, eq2, eq3], (x_sym, y_sym, z_sym))

        # Si hay una solución única, graficarla
        if solution:
            sol = list(solution)[0]
            if len(sol.free_symbols) == 0:
                # Punto de intersección único
                ax.scatter(float(sol[0]), float(sol[1]), float(sol[2]), color='black', s=100, label='Punto de intersección')
                print(f"Punto de intersección: ({sol[0]}, {sol[1]}, {sol[2]})")
            else:
                print("Los planos se intersectan en una línea.")
        else:
            print("No hay solución.")

        # Mostrar leyenda y gráfico
        plt.legend()
        plt.show()

# Función para actualizar la rotación en tiempo real
def update_rotation(change):
    with output:
        clear_output(wait=True)
        plot_planes()

# Conectar el botón de graficar con la función
plot_button.on_click(plot_planes)
rotation_slider.observe(update_rotation, names='value')

# Mostrar los widgets
display(widgets.VBox([equation1_box, equation2_box, equation3_box, rotation_slider, plot_button, output]))
